<a href="https://colab.research.google.com/github/peterlhn/Google-Colab/blob/master/Matrix_Practices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
from scipy.stats import norm
import pandas as pd

**Create class "CASE" to detemine input**

In [36]:
class single_case:
  def __init__(self, id, mean, sd, pub_p, pub_v, retailer_p, retailer_s):
    self.id = id
    self.mean = mean
    self.sd = sd
    self.pub_p = pub_p
    self.pub_v = pub_v
    self. retailer_p = retailer_p
    self.retailer_s = retailer_s
  

**Create method to calculate the outcome**

In [84]:
# Method - Find Revenue Sharing affecting Supply Chain Profit
def best_revenue_share(case, f):
    case_id = case.id
    f = f

    mean = case.mean
    sd = case.sd

    pub_p = case.pub_p
    pub_v = case.pub_v
    retailer_p = case.retailer_p
    retailer_c = pub_p
    retailer_s = case.retailer_s

    # ------ Cost of Understocking / Overstocking ------- #
    cu = (1-f)*retailer_p - retailer_c
    co = retailer_c - retailer_s
    # Critical Fractile = Cu/(Cu+Co)
    cf = cu/(cu+co)
    # Optimal Order Quantity (Q*) 
    Q = norm.ppf(cf, loc=mean, scale=sd)

    # -------  Number of Under / Overstocking -------- # 
    num_diff = sd *  norm(loc = 0 , scale = 1).pdf((Q-mean)/sd)

    # Understocking # 
    e_under = 1 - norm(loc = mean , scale = sd).cdf(Q)
    num_under_ind = (mean - Q)*e_under 
    num_under_sum = num_diff + num_under_ind
    
    # Overstocking # 
    e_over = norm(loc = mean , scale = sd).cdf(Q)
    num_over_ind =  (Q-mean)*e_over
    num_over_sum = num_diff + num_over_ind

    # Actual Sales #
    act_sale = Q - num_over_sum

    # ------- Retailer Profit ---------- #
    sales_retailer = round((1-f)*retailer_p * act_sale,0)       # (1-f)*price * Actual Sales #
    overstock_sales_retailer = round(num_over_sum * retailer_s,0)        # Overstock # X Salvage value
    cost_retailer = round(retailer_c * Q,0)         # Optimal Order Quantity * Cost

    profit_retailer = sales_retailer + overstock_sales_retailer - cost_retailer

    # ------ Publisher Profit -------- #
    sales_publisher = round(pub_p * Q,0)                           # price * Q
    share_revenue_publisher = round(f * retailer_p * act_sale,0)   # f * p * act_sale
    cost_publisher = round(pub_v * Q,0)                            # Optimal Order Quantity * Cost

    profit_publisher = sales_publisher + share_revenue_publisher - cost_publisher      

    # ------ Supplychain Profit -------- #
    supply_chain_profit = profit_publisher + profit_retailer

    return [case_id, f, cu, co, cf, Q, profit_retailer, act_sale,num_under_sum, num_over_sum, profit_publisher, supply_chain_profit]


In [82]:
def multi_cases(case,step):
  case_ultra_low = single_case(case.id+"_ultra_low", case.mean, case.sd, case.pub_p*(1-2*step), case.pub_v, case.retailer_p, case.retailer_s)
  case_low = single_case(case.id+"_low", case.mean, case.sd, case.pub_p*(1-step), case.pub_v, case.retailer_p, case.retailer_s)
  case_high = single_case(case.id+"_high", case.mean, case.sd, case.pub_p*(1+step), case.pub_v, case.retailer_p, case.retailer_s)
  case_ultra_high = single_case(case.id+"_ultra_high", case.mean, case.sd, case.pub_p*(1+2*step), case.pub_v, case.retailer_p, case.retailer_s)
  return [case, case_ultra_low, case_low, case_high, case_ultra_high]


**Create FOUR case, assign to function**

In [89]:
# Create a cases 
case_org = single_case('Ben', 20000, 4000, 10, 1, 24, 3)
cases_list = multi_cases(case_org,0.25)   # [case, case_ultra_low, case_low, case_high, case_ultra_high]
# Assign these cases into calculation method
for i in cases_list:
  result = best_revenue_share(i,0.1)
  print(result)


['Ben', 0.1, 11.600000000000001, 7, 0.6236559139784946, 21260.387126500955, 203756.0, 18955.85468192033, 1044.1453180796718, 2304.5324445806264, 236838.0, 440594.0]
['Ben_ultra_low', 0.1, 16.6, 2.0, 0.8924731182795699, 24959.152588395344, 318237.0, 19793.298750712835, 206.70124928716393, 5165.853837682507, 147341.0, 465578.0]
['Ben_low', 0.1, 14.100000000000001, 4.5, 0.7580645161290323, 22800.36085043027, 258769.0, 19428.56980036324, 571.4301996367611, 3371.7910500670296, 194832.0, 453601.0]
['Ben_high', 0.1, 9.100000000000001, 9.5, 0.48924731182795705, 19892.174974296708, 152329.0, 18349.738626001184, 1650.2613739988162, 1542.4363482955239, 272799.0, 425128.0]
['Ben_ultra_high', 0.1, 6.600000000000001, 12.0, 0.35483870967741943, 18510.842558139237, 104306.0, 17550.32564146596, 2449.674358534042, 960.5169166732784, 301273.0, 405579.0]


In [90]:
data = []
for case in cases_list:
  for i in range(0,100,5):
    x = best_revenue_share(case, i/100) 
    data.append(x)


In [91]:
df = pd.DataFrame(data, columns=['Case ID','Revenue Share','Cost of Understock','Cost of Overstock','Critical Fractile', 'Optimal Order Quantity', 'Actual Sales', 'Understock', 'Overstock', 'Retailers Profit', 'Publisher Profit', 'Supplychain Profit']).round(2)

In [92]:
df

,Case ID,Revenue Share,Cost of Understock,Cost of Overstock,Critical Fractile,Optimal Order Quantity,Actual Sales,Understock,Overstock,Retailers Profit,Publisher Profit,Supplychain Profit
0,Ben,0.00,14.0,7.0,0.67,21722.91,249458.0,19119.90,880.10,2603.01,195506.0,444964.0
1,Ben,0.05,12.8,7.0,0.65,21503.17,226557.0,19044.45,955.55,2458.73,216382.0,442939.0
2,Ben,0.10,11.6,7.0,0.62,21260.39,203756.0,18955.85,1044.15,2304.53,236838.0,440594.0
3,Ben,0.15,10.4,7.0,0.60,20989.61,181070.0,18850.44,1149.56,2139.16,256768.0,437838.0
4,Ben,0.20,9.2,7.0,0.57,20684.13,158524.0,18723.01,1276.99,1961.12,276027.0,434551.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Ben_ultra_high,0.75,-9.0,12.0,-3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Ben_ultra_high,0.80,-10.2,12.0,-5.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Ben_ultra_high,0.85,-11.4,12.0,-19.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Ben_ultra_high,0.90,-12.6,12.0,21.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# Max "Supplychain Profit" row
df.iloc[df['Supplychain Profit'].argmax()]

Case ID                   Ben_ultra_low
Revenue Share                         0
Cost of Understock                   19
Cost of Overstock                     2
Critical Fractile                   0.9
Optimal Order Quantity          25236.7
Actual Sales                     365777
Understock                      19821.4
Overstock                        178.59
Retailers Profit                5415.28
Publisher Profit                 100946
Supplychain Profit               466723
Name: 20, dtype: object

In [95]:
# Min "Supplychain Profit" row
df.iloc[df['Supplychain Profit'].argmin()]

Case ID                   Ben_ultra_high
Revenue Share                       0.35
Cost of Understock                   0.6
Cost of Overstock                     12
Critical Fractile                   0.05
Optimal Order Quantity           13326.4
Actual Sales                        7000
Understock                       13247.5
Overstock                        6752.54
Retailers Profit                   78.98
Publisher Profit                  297850
Supplychain Profit                304850
Name: 87, dtype: object

###**Single Calculation**

In [29]:
import numpy as np

**Model Inputs**

In [30]:
mean = 20000
sd = 4000

# Publisher
pub_p = 9.5
pub_v = 1

# Retailer
f = 0.1      # # Revenue Sharing %
retailer_p = 24
retailer_c = pub_p
retailer_s = 3

**Critical Numbers**

In [31]:
# Cost of Understocking, Cu = (1-f)*p - c
cu = (1-f)*retailer_p - retailer_c
# Cost of Overstocking, Co = c - s
co = retailer_c - retailer_s

# Critical Fractile = Cu/(Cu+Co)
# Chance of successfuly meet the demand 
cf = cu/(cu+co)

# Optimal Order Quantity (Q*) 
# NORM.INV in Excel -> loc = mean, scale = sd
Q = norm.ppf(cf, loc=mean, scale=sd)


print(f"Cost of Understocking (Cu) is {round(cu,1)}")
print(f"Cost of Overstocking (Co) is {round(co,1)}")

print(f"Optimal Order Quantity is {round(Q,0)}, given Critical Fractile is {round(cf,10)}")

Cost of Understocking (Cu) is 12.1
Cost of Overstocking (Co) is 6.5
Optimal Order Quantity is 21547.0, given Critical Fractile is 0.6505376344


**Expected Number of Over & Understock**

In [32]:
# Probability that the demand will be under / over Optimal Order Quantity
# cdf = cumulative distribution function.
num_diff = sd *  norm(loc = 0 , scale = 1).pdf((Q-mean)/sd)

# NORM.DIST(20000, 4000, 21547,1) = 0.65 =CF
# pdf = Probability density function.
# num_under = (mean-sd)*(1-NORM.DIST(Q*,mean,sd,1))
e_under = 1 - norm(loc = mean , scale = sd).cdf(Q)
num_under_ind = (mean - Q)*e_under 
num_under_sum = num_diff + num_under_ind
print(f'Understock is {round(num_under_ind,0)} + {round(num_diff,0)} = {round(num_under_sum,0)}, given the standard index is {round(e_under,2)}')


# num_under = (mean-sd)*(1-NORM.DIST(Q*,mean,sd,1))
e_over = norm(loc = mean , scale = sd).cdf(Q)
num_over_ind =  (Q-mean)*e_over
num_over_sum = num_diff + num_over_ind

print(f'Overstock is {round(num_over_ind,0)} + {round(num_diff,0)} = {round(num_over_sum,0)}, given the standard index is {round(e_over,2)}')

# Actual Sales = Q* - Overstock
act_sale = Q - num_over_sum
print(f'Actual Sales at P is {round(act_sale,0)}')

Understock is -541.0 + 1481.0 = 940.0, given the standard index is 0.35
Overstock is 1006.0 + 1481.0 = 2487.0, given the standard index is 0.65
Actual Sales at P is 19060.0


**Revenue, Cost, and Profit**

**Retailer**

In [33]:
# (1-f)*price * Actual Sales #
sales_retailer = round((1-f)*retailer_p * act_sale,0)

# Overstock # X Salvage value
overstock_sales_retailer = round(num_over_sum * retailer_s,0)

# Optimal Order Quantity * Cost
cost_retailer = round(retailer_c * Q,0)

# Profit = Sum of 3 above 
profit_retailer = sales_retailer + overstock_sales_retailer - cost_retailer

print(f'{sales_retailer}, {overstock_sales_retailer}, {cost_retailer}')
print(f'Retailers profit is {profit_retailer}')

411693.0, 7462.0, 204697.0
Retailers profit is 214458.0


**Manufacturer**

In [34]:
# price * Q
sales_publisher = round(pub_p * Q,0)

# Revenue Share = f * p * act_sale
share_revenue_publisher = round(f * retailer_p * act_sale,0)

# Optimal Order Quantity * Cost
cost_publisher = round(pub_v * Q,0)

# Profit = Sum of 3 above 
profit_publisher = sales_publisher + share_revenue_publisher - cost_publisher

print(f'{sales_publisher}, {share_revenue_publisher}, {cost_publisher}')
print(f'Retailers profit is {profit_publisher}')

204697.0, 45744.0, 21547.0
Retailers profit is 228894.0


###**Reference**